# BIKE SHARING CASE STUDY


## Problem Statement:

A US bike-sharing provider BoomBikes has recently suffered considerable dips in their revenues due to 
the ongoing Corona pandemic. The company is finding it very difficult to sustain in the current market scenario.
So, it has decided to come up with a mindful business plan to be able to accelerate its revenue as soon as the 
ongoing lockdown comes to an end, and the economy restores to a healthy state.

Specifically, they want to understand the factors affecting the demand for these shared bikes in the American market.
The company wants to know:

- Which variables are significant in predicting the demand for shared bikes.

- How well those variables describe the bike demands



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**LIBRARIES**

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score

# Reading and Understanding the Data

In [ ]:
bike_sharing=pd.read_csv("/kaggle/input/boombikes/day.csv")
bike_sharing.head()

In [ ]:
bike_sharing.shape

In [ ]:
bike_sharing.info()

In [ ]:
bike_sharing.describe()

# Visualising the Data



In [ ]:
plt.figure(figsize=(16,10))
sns.pairplot(bike_sharing)
plt.show()

# Data Preparation

here, to create dummy variables,columns that are represnting month,weekday,season,weathersit are in 
numerical values but by chnaging it into categorical value it will recognize the dummy varibles.

In [ ]:
bike_sharing['season']=bike_sharing['season'].map({1:'spring', 2:'summer', 3:'fall', 4:'winter'})
bike_sharing.head()

In [ ]:
dum_season = pd.get_dummies(bike_sharing['season'],drop_first = True)
dum_season.head()

In [ ]:
bike_sharing['weathersit']=bike_sharing['weathersit'].map({1:'clear', 2:'mist', 3:'lightrain', 4:'heavyrain'})
bike_sharing.head()

In [ ]:
dum_weathersit = pd.get_dummies(bike_sharing['weathersit'],drop_first = True)
dum_weathersit.head()

In [ ]:
bike_sharing['mnth']=bike_sharing['mnth'].map({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'})
bike_sharing.head()

In [ ]:
dum_mnth = pd.get_dummies(bike_sharing['mnth'],drop_first = True)
dum_mnth.head()

In [ ]:
bike_sharing['weekday']=bike_sharing['weekday'].map({0:'Sun',1:'Mon', 2:'Tue', 3:'Wed', 4:'Thu',5:'Fri',6:'Sat'})
bike_sharing.head()

In [ ]:
dum_weekday = pd.get_dummies(bike_sharing['weekday'],drop_first = True)
dum_weekday.head()

In [ ]:
# concating the dummy varibles into the dataset.
bike_sharing = pd.concat([bike_sharing, dum_season,dum_weathersit,dum_mnth,dum_weekday], axis = 1)
bike_sharing.head()

In [ ]:
# here we also drop registered and casual varibles as it is higly correlated with target varible count

bike_sharing.drop(['season','dteday','mnth','weekday','casual','registered','instant','weathersit','atemp'], axis = 1, inplace = True)
bike_sharing.head()

# Splitting the Data into Training and Testing Sets

In [ ]:
bike_sharing_train, bike_sharing_test = train_test_split(bike_sharing, train_size = 0.7, test_size = 0.3, random_state = 100)

In [ ]:
bike_sharing_test.shape

In [ ]:
bike_sharing_train.shape

In [ ]:
scaler = MinMaxScaler()

In [ ]:
# scaling the train dataset
num_vars = ['temp','hum','windspeed','cnt']

bike_sharing_train[num_vars] = scaler.fit_transform(bike_sharing_train[num_vars])

In [ ]:
bike_sharing_train.head()

In [ ]:
bike_sharing_train.describe()

In [ ]:
plt.figure(figsize = (16, 10))
sns.heatmap(bike_sharing_train.corr(), annot = True, cmap="YlGnBu")
plt.show()

**Inference:**
    year and temperatue variable is highly correlated

# Dividing into X and Y sets for the model building

In [ ]:
y_train = bike_sharing_train.pop('cnt') # target variable
X_train = bike_sharing_train


# Building a linear model 
**using RFE(Recursive Feature Elimination)**

In [ ]:
lr=LinearRegression()
lr.fit(X_train,y_train)
rfe=RFE(lr,15)
rfe=rfe.fit(X_train,y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
col=X_train.columns[rfe.support_]
col

In [ ]:
X_train.columns[~rfe.support_]

In [ ]:
# Assigning the feature variables having True value to X
X_train_rfe=X_train[col]

In [ ]:
# Adding constants to X_train
X_train_lr=sm.add_constant(X_train_rfe)

In [ ]:
# Create a first fitted model
lr=sm.OLS(y_train,X_train_lr).fit()

In [ ]:
# Print a summary of the linear regression model obtained
print(lr.summary())

In [ ]:
# Check for the VIF values of the feature variables. 
vif = pd.DataFrame()
X=X_train_rfe
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Dropping highly correlated variables 
X_train_new=X_train_rfe.drop(['hum'],axis=1)

In [ ]:
X_train_lr=sm.add_constant(X_train_new)
lr_1=sm.OLS(y_train,X_train_lr).fit()

In [ ]:
# Print a summary of the linear regression model obtained
print(lr_1.summary())

In [ ]:
# Check for the VIF values of the feature variables. 
vif = pd.DataFrame()
X=X_train_new
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Dropping highly correlated variables as temp has high coefficient so we drop next variable with high VIF.

X_train_new1=X_train_new.drop(['workingday'],axis=1)

In [ ]:
X_train_lr=sm.add_constant(X_train_new1)
lr_2=sm.OLS(y_train,X_train_lr).fit()

In [ ]:
# Print a summary of the linear regression model obtained
print(lr_2.summary())

In [ ]:
# Check for the VIF values of the feature variables. 
vif = pd.DataFrame()
X=X_train_new1
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X_train_new2=X_train_new1.drop(['Sat'],axis=1)

In [ ]:
X_train_lr=sm.add_constant(X_train_new2)
lr_3=sm.OLS(y_train,X_train_lr).fit()

In [ ]:
# Print a summary of the linear regression model obtained
print(lr_3.summary())

In [ ]:
# Check for the VIF values of the feature variables. 
vif = pd.DataFrame()
X=X_train_new2
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X_train_new3=X_train_new2.drop(['Jan'],axis=1)

In [ ]:
X_train_lr=sm.add_constant(X_train_new3)
lr_4=sm.OLS(y_train,X_train_lr).fit()

In [ ]:
# Print a summary of the linear regression model obtained
print(lr_4.summary())

In [ ]:
# Check for the VIF values of the feature variables. 
vif = pd.DataFrame()
X=X_train_new3
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X_train_new4=X_train_new3.drop(['summer'],axis=1)

In [ ]:
X_train_lr=sm.add_constant(X_train_new4)
lr_5=sm.OLS(y_train,X_train_lr).fit()

In [ ]:
# Print a summary of the linear regression model obtained
print(lr_5.summary())

In [ ]:
# Check for the VIF values of the feature variables. 
vif = pd.DataFrame()
X=X_train_new4
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X_train_new5=X_train_new4.drop(['winter'],axis=1)

In [ ]:
X_train_lr=sm.add_constant(X_train_new5)
lr_6=sm.OLS(y_train,X_train_lr).fit()

In [ ]:
# Print a summary of the linear regression model obtained
print(lr_6.summary())

In [ ]:
# Check for the VIF values of the feature variables. 
vif = pd.DataFrame()
X=X_train_new5
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X_train_new6=X_train_new5.drop(['Sep'],axis=1)

In [ ]:
X_train_lr=sm.add_constant(X_train_new6)
lr_7=sm.OLS(y_train,X_train_lr).fit()

In [ ]:
# Print a summary of the linear regression model obtained
print(lr_7.summary())

In [ ]:
# Check for the VIF values of the feature variables. 
vif = pd.DataFrame()
X=X_train_new6
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

# Residual Analysis of the train data

So, now to check if the error terms are also normally distributed (which is infact, one of the major assumptions of linear regression), let us plot the histogram of the error terms and see what it looks like.

In [ ]:
y_train_cnt = lr_7.predict(X_train_lr)

In [ ]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_cnt), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                   
plt.xlabel('Errors', fontsize = 18)  

# Making Predictions Using the Final Model

Now that we have fitted the model and checked the normality of error terms,
it's time to go ahead and make predictions using the final, i.e. seventh model.

In [ ]:
num_vars = ['temp','hum','windspeed','cnt']

bike_sharing_test[num_vars] = scaler.transform(bike_sharing_test[num_vars])

In [ ]:
bike_sharing_test.describe()

# Dividing into X_test and y_test

In [ ]:
y_test = bike_sharing_test.pop('cnt')
X_test = bike_sharing_test

In [ ]:
# Adding constant variable to test dataframe
X_test_lr = sm.add_constant(X_test)
X_test_lr.head()

In [ ]:
X_test_lr=X_test_lr.loc[:,['const','yr','holiday','temp','windspeed','spring','lightrain','mist','Jul']]
X_test_lr

In [ ]:
# Making predictions using the seventh model
y_pred_cnt = lr_7.predict(X_test_lr)

# Model Evaluation
Let's now plot the graph for actual versus predicted values.

In [ ]:
fig = plt.figure()
plt.scatter(y_test, y_pred_cnt)
fig.suptitle('y_test vs y_pred', fontsize = 20)              
plt.xlabel('y_test', fontsize = 18)                          
plt.ylabel('y_pred', fontsize = 16) 

We can see that the equation of our best fitted line is:

$ price = 0.234  \times year -0.867  \times holiday +0.408 \times temperature -0.159 \times windspeed -0.146 \times spring -0.273 \times lighttrain -0.080 \times mist + -0.085 \times Jul  $

In [ ]:
r2=r2_score(y_test, y_pred_cnt)
r2

**r2 of train set is 0.824**